In [163]:
import os
import pandas as pd
import numpy as np

cwd = os.getcwd()
datadir = '/'.join(cwd.split('/')[0:-1])+ '/'
cwd
df = pd.read_csv('data.csv')
df = df.set_index('defendantid')

# df.dropna(axis = 0,inplace = True)

# IMPORTANT - dropna dropped 48k rows from our original dataset of 100k.
df.columns[df.isnull().any()]
# So columns which have null values: ['race', 'category', 'MajorGroupDetail', 'AgeAtFirst', 'Age',
# 'CareerLength', 'CvtDensity']

# count of null values in each column [AgeAtFirst,CareerLength and CvtDensity have > 40k NA values.]
# Upon checking the feature importance values from the random forest built in the exploration file, 
# it seems these 3 are the most important variables.

# We'll need to chek the feature importance to decide whether to keep those instances / transform them or not.
df[['race','category','MajorGroupDetail','AgeAtFirst','Age','CareerLength','CvtDensity']].isnull().sum()

df[df.columns[-10:]]

df.drop(['NewViolArrest36','NewMisdArrest36','NewFelArrest36','NewArrest36'],axis = 1, inplace = True)
df.columns

#created binary target variable
df['NewArrest36Type_binary'] = df['NewArrest36Type']

#classified target variable from multiclass to binary
df.NewArrest36Type_binary[df.NewArrest36Type_binary == 'None'] = 0 
df.NewArrest36Type_binary[df.NewArrest36Type_binary == 'Misdemeanor'] = 1 
df.NewArrest36Type_binary[df.NewArrest36Type_binary == 'Non-Violent Felony'] = 1 
df.NewArrest36Type_binary[df.NewArrest36Type_binary == 'Violent Felony'] = 1 
df.head()

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

,gender,race,category,MajorGroupDetail,AgeAtFirst,OrderProtect,Housing,Age,MisdConviction,FelonyConviction,...,Jail10,Prison10,CareerLength,CvtDensity,NewArrests36,NewFelArrests36,NewMisdArrests36,NewViolArrests36,NewArrest36Type,NewArrest36Type_binary
defendantid,,,,,,,,,,,,,,,,,,,,,
15372,M,Asian/Pacific Islander,Misdemeanor,UGV,NaN,0,0,32.0,0,0,...,0,0,NaN,NaN,0,0,0,0,None,0
16191,F,Black,Misdemeanor,Forged Instrument,30.0,0,0,42.0,0,0,...,0,0,12.0,0.000000,0,0,0,0,None,0
24,M,Black,Misdemeanor,Marijuana,23.0,1,0,30.0,1,0,...,4,0,8.0,0.866025,1,0,1,0,Misdemeanor,1
959,M,Black-Hispanic,Misdemeanor,Drugs,33.0,0,1,54.0,1,1,...,11,0,23.0,1.693979,18,1,17,0,Non-Violent Felony,1
1596,M,Black,Felony,Grand Larceny,18.0,0,0,24.0,1,1,...,2,0,7.0,0.845154,1,1,0,0,Non-Violent Felony,1


In [164]:
df.columns[df.isnull().any()]

Index(['race', 'category', 'MajorGroupDetail', 'AgeAtFirst', 'Age',
       'CareerLength', 'CvtDensity'],
      dtype='object')

In [165]:
df.race.unique()

array(['Asian/Pacific Islander', 'Black', 'Black-Hispanic',
       'White-Hispanic', 'White', 'Unsure', 'Unknown',
       'American Indian/Alaskan Native', nan], dtype=object)

In [166]:
df.gender[df.gender == 'M'] = 0 
df.gender[df.gender == 'F'] = 1 

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [167]:
df.drop(['NewViolArrests36','NewMisdArrests36','NewFelArrests36','NewArrests36','NewArrest36Type'],axis = 1, inplace = True)


In [168]:
# coverting other variables from multi class to numeric
from sklearn.preprocessing import LabelEncoder
df.head()
label_encoder = LabelEncoder()
df['race_encoded'] = label_encoder.fit_transform(df.race.fillna('0'))
df['category_encoded'] = label_encoder.fit_transform(df.category.fillna('0'))
df['MajorGroupDetail_encoded'] = label_encoder.fit_transform(df.MajorGroupDetail.fillna('0'))
# to check the mapping
list(label_encoder.inverse_transform([2,1,2]))

df.head()
df.drop(['race','category','MajorGroupDetail'],axis = 1, inplace = True)



In [169]:
cols = df.columns.tolist()
cols = cols[-3:] + cols[:-3]
cols
df = df[cols]
df.head()

,race_encoded,category_encoded,MajorGroupDetail_encoded,gender,AgeAtFirst,OrderProtect,Housing,Age,MisdConviction,FelonyConviction,...,Prison,Incarcerations5,Jail5,Prison5,Incarcerations10,Jail10,Prison10,CareerLength,CvtDensity,NewArrest36Type_binary
defendantid,,,,,,,,,,,,,,,,,,,,,
15372,2,3,38,0,NaN,0,0,32.0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,0
16191,3,3,17,1,30.0,0,0,42.0,0,0,...,0,0,0,0,0,0,0,12.0,0.000000,0
24,3,3,23,0,23.0,1,0,30.0,1,0,...,0,4,4,0,4,4,0,8.0,0.866025,1
959,4,3,16,0,33.0,0,1,54.0,1,1,...,1,4,4,0,11,11,0,23.0,1.693979,1
1596,3,1,20,0,18.0,0,0,24.0,1,1,...,0,2,2,0,2,2,0,7.0,0.845154,1


In [130]:
# #normalize the feature columns to set the mean = 0 and std = 1
# from sklearn.preprocessing import StandardScaler

# featurecolumns = df.columns[:-1]
# scaler = StandardScaler()
# df[featurecolumns] = scaler.fit_transform(df[featurecolumns])


'object'